In [19]:
import cv2
import mediapipe as mp
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [20]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
cap = cv2.VideoCapture (0)
height= cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width= cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get (cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('leg_position1.mp4', cv2.VideoWriter_fourcc('P','I', 'M','1'), fps, (int(width), int(height)))
while cap.isOpened():
    ret, frame = cap.read()
    try:
        cv2.imshow('leg_position1', frame)
        videoWriter.write(frame)
    except  Exception as e:
        break
    if cv2.waitKey (10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

In [21]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
with open('feature_leg_position.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
def export_landmark(results, action):
    try:
        keypoints = [action]  # Initialize the list with the action label

        # Extract pose landmarks and append them to the keypoints list
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        # Open the CSV file in 'a' (append) mode and write the keypoints
        with open('feature_leg_position.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(f"Error in export_landmark: {e}")

In [ ]:

# Create a VideoCapture object to open the video file
video_path = 'C:/Projects/FinalProject/excesise_proj/leg_exces/leg_position.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Initialize a variable to keep track of the current pose
current_pose = None

# Create a dictionary to map keys to pose labels
key_to_pose = {
    ord('a'): 'small',
    ord('s'): 'mid',
    ord('d'): 'wide'
}

# Main processing loop
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break  # Exit the loop when there are no more frames

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform pose detection
        results = pose.process(image)

        # Convert back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and lines on the frame
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Check for key presses and export landmarks
        key = cv2.waitKey(1)
        if key in key_to_pose:
            current_pose = key_to_pose[key]
            #print(current_pose)
            export_landmark(results, current_pose)

        # Display the frame
        cv2.imshow('webcam feed', image)

        # Check for 'q' key to quit
        if key == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [81]:
df = pd.read_csv('C:/Projects/FinalProject/excesise_proj/leg_exces/feature_leg_position.csv')

In [82]:
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,wide,0.655026,0.045260,-0.589442,0.999983,0.664644,0.027130,-0.575842,0.999960,0.669763,...,0.182368,0.737731,0.743783,0.880988,-0.017547,0.961876,0.554220,0.869039,0.003634,0.954608
1,wide,0.663030,0.048134,-0.624495,0.999986,0.672878,0.028771,-0.608860,0.999963,0.678245,...,0.247037,0.735461,0.731904,0.880210,0.051311,0.957008,0.536151,0.859882,0.075392,0.952495
2,wide,0.661599,0.049464,-0.633611,0.999988,0.671619,0.029611,-0.616365,0.999967,0.676916,...,0.260525,0.761741,0.736948,0.880240,0.040138,0.960436,0.542046,0.853574,0.091661,0.956912
3,wide,0.661172,0.050654,-0.618133,0.999989,0.671022,0.030381,-0.603611,0.999967,0.676143,...,0.241067,0.765429,0.733758,0.880682,0.046070,0.960120,0.548025,0.862142,0.057138,0.956754
4,wide,0.660935,0.049306,-0.558362,0.999988,0.670924,0.030153,-0.543855,0.999962,0.675930,...,0.171301,0.749507,0.733039,0.880386,-0.025799,0.954908,0.542212,0.854872,-0.007017,0.952080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,small,0.687472,0.033901,-0.506831,0.999989,0.697686,0.016992,-0.492989,0.999959,0.702554,...,0.189015,0.630392,0.686979,0.864268,0.044717,0.942550,0.631070,0.851062,0.036421,0.921771
519,small,0.687045,0.034629,-0.445158,0.999988,0.696739,0.017472,-0.435109,0.999955,0.701611,...,0.139999,0.625401,0.687378,0.861439,0.009014,0.941465,0.631360,0.848404,0.008940,0.920657
520,small,0.686708,0.028670,-0.534402,0.999987,0.696151,0.013826,-0.515486,0.999951,0.701136,...,0.188720,0.626154,0.686726,0.863504,0.048705,0.941926,0.635794,0.846513,0.036890,0.921364
521,small,0.684887,0.033988,-0.559904,0.999987,0.695049,0.016111,-0.548919,0.999949,0.700176,...,0.212547,0.627553,0.685103,0.863187,0.065961,0.941352,0.636388,0.854149,0.060205,0.921446


In [83]:
x = df.drop('class', axis=1)
y=df['class']

In [84]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=1234)

In [76]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [85]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [86]:
y_pred = model.predict(x_test)

In [87]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.90
              precision    recall  f1-score   support

         mid       0.92      0.87      0.89        75
       small       0.87      0.95      0.91        42
        wide       0.90      0.90      0.90        40

    accuracy                           0.90       157
   macro avg       0.90      0.91      0.90       157
weighted avg       0.90      0.90      0.90       157

[[65  6  4]
 [ 2 40  0]
 [ 4  0 36]]


In [89]:
import pickle
# Save the model to a file

model_file = 'C:/Projects/FinalProject/excesise_proj/leg_exces/leg_position_knn.joblib'
with open(model_file, 'wb') as file:
    pickle.dump(model, file)
print(f"Model saved as {model_file}")

Model saved as C:/Users/shana/OneDrive/Desktop/excesise_proj/leg_exces/leg_position_knn.joblib


In [71]:
from joblib import dump
model_file = 'C:/Projects/FinalProject/excesise_proj/leg_exces/leg_position_V3.joblib'
dump(model, model_file)

['C:/Users/shana/OneDrive/Desktop/excesise_proj/leg_exces/leg_position_V3.joblib']

In [88]:
cap = cv2.VideoCapture(0)
state = ''

# Mapping of classes to actions
class_to_action = {
    'nutral': 'nutral',
    'mid': 'mid',
    'wide': 'wide'
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            x = pd.DataFrame([row], columns=landmarks[1:])
            state_class = model.predict(x)[0]
            state_prob = model.predict_proba(x)[0]
            #print(f"State: {state_class}, State Prob: {state_prob}")

            if state_class in class_to_action and state_prob[state_prob.argmax()] >= 0.7:
                state = class_to_action[state_class]

            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, state_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(round(state_prob[np.argmax(state_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow('webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


In [ ]:
loaded_model = load(model_file)

In [ ]:
y_pred_loaded = loaded_model.predict(x_test)
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print(f'Loaded Model Accuracy: {accuracy_loaded:.2f}')

In [ ]:
cap = cv2.VideoCapture(0)
state = ''

# Mapping of classes to actions
class_to_action = {
    'nutral': 'nutral',
    'mid': 'mid',
    'wide': 'wide'
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            x = pd.DataFrame([row], columns=landmarks[1:])
            state_class = loaded_model.predict(x)[0]
            state_prob = loaded_model.predict_proba(x)[0]
            #print(f"State: {state_class}, State Prob: {state_prob}")

            if state_class in class_to_action and state_prob[state_prob.argmax()] >= 0.7:
                state = class_to_action[state_class]

            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, state_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(round(state_prob[np.argmax(state_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow('webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
